# Bresenham’s line
[Bresenham’s line](https://en.wikipedia.org/wiki/Bresenham%27s_line_algorithm) is another beautiful algorithm in this series.

How hard could it be to draw a line? Well, when your resources are limited — I mean, limited — it is not that easy. And there is a beautiful set of equations to draw a line and spare on resources as much as possible.

![day73-bresenhamsline_1](resource/day73-bresenhamsline_1.png)

Assume we draw a line from left to right with x-axis as major.

$$ y=\frac{\Delta y}{\Delta x}x+b $$
$$ \Delta x \geq \Delta y \geq 0 $$

Given we drew a point at `[x, y]`, the next point will be either `[x+1, y]` or `[x+1, y+1]`. We can find a difference between these two points and the real point.

$$ d_1 = y-y_i=\frac{\Delta y}{\Delta x}(x_i+1)+b-y_i $$
$$ d_2 = y_1+1-y=y_i+1-\frac{\Delta y}{\Delta x}(x_i+1)-b $$
$$ d = d_1 - d_2 = 2\frac{\Delta y}{\Delta x}(x_i+1)-2y_i+2b+1 $$

Notice that the sign of `d` controls which of the two points is closer to the desired point. And since the sign is all that matters, we can multiply `d` by a positive number to convert the equation into integer arithmetic.

$$ d\Delta x= 2d\Delta y(x_i+1)-2y_i\Delta x +\Delta x(2b-1) $$


$$ p_i = d \Delta x = 2\Delta y x_i-2y_i\Delta x+2\Delta y+\Delta x(2b-1) $$
$$ p_{i+1} = 2\Delta y x_{i+1}-2y_{i+1}\Delta x+const. $$
$$ p_{i+1} = p_i + 2\Delta y-2(y_{i+1}-y_i)\Delta x$$

Variable `p` can be expressed recursively and gives us an integer predictor of the next point.

If `p` is positive, `y` is increased; If it is negative, `y` remains unchanged.

$$ p_i \leq 0:p{i+1}=p_i+2\Delta y $$
$$ p_i > 0:p{i+1}=p_i+2\Delta y -2\Delta x $$

Notice there is no multiplication and no division in the algorithm, adding and subtracting integers is all we need. Pretty cool, isn’t it?

![day73-bresenhamsline_2](resource/day73-bresenhamsline_2.png)

In [1]:
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.palettes import Set3_12

output_notebook()

Loading BokehJS ...

## algorithm

In [2]:
def bresenham(x0, y0, x1, y1, color, canvas):
    # swap major and minor axis
    if abs(y0 - y1) > abs(x0 - x1):
        x0, y0, x1, y1 = y0, x0, y1, x1
        canvas = canvas.T
    
    # swap x-axis direction
    if x0 > x1:
        x0, y0, x1, y1 = x1, y1, x0, y0
        
    # initialize
    dx, dy = abs(x0 - x1), abs(y0 - y1)
    dz = 1 if y0 <= y1 else -1
    p = 2 * dy - dx
    
    # draw
    while x0 <= x1:
        canvas[y0, x0] = color

        if p > 0:
            p -= 2 * dx
            y0 += dz
        p += 2 * dy
        x0 += 1

## plot

In [3]:
def plot(canvas):
    w, h = canvas.shape
    
    plot = figure(x_range=(0, w - 1), y_range=(0, h - 1))
    plot.grid.visible = False
    plot.axis.visible = False
    image = plot.image([canvas], 
                       x=0, y=0, dw=w - 1, dh=h - 1,
                       palette=['#000000'] + Set3_12)
    
    show(plot)

In [12]:
n = 50
canvas = np.zeros((n, n), dtype=int)

# 8 basic lines
m = n // 2
bresenham(m, m, n-1, m+10, 1, canvas)
bresenham(m, m, n-1, m-10, 2, canvas)
bresenham(m, m, m+10, n-1, 3, canvas)
bresenham(m, m, m+10, 0, 4, canvas)
bresenham(0, m+10, m, m, 5, canvas)
bresenham(0, m-10, m, m, 6, canvas)
bresenham(m-10, n-1, m, m, 7, canvas)
bresenham(m-10, 0, m, m, 8, canvas)
bresenham(m, m, m, m, 0, canvas)

In [13]:
plot(canvas)

In [6]:
n = 100
canvas = np.zeros((n, n), dtype=int)

for i in range(10):
    x = np.random.randint(0, n, 4)
    bresenham(*x, i + 1, canvas)

In [7]:
plot(canvas)

In [8]:
n = 200
canvas = np.zeros((n, n), dtype=int)

for i in range(n):
    bresenham(0, 0, i, n - 1, i % 10 + 1, canvas)
    bresenham(0, 0, n - 1, i, i % 10 + 1, canvas)

In [9]:
plot(canvas)